## Simple TensorFlow chatbot
#### Author: Sokolov Alexander
This notebook was created for Concepta international e-commerce agency

08.06.2022

## Contents:
* [Pre-work](#pre)
* [Intents preparation](#prep)
* [Training](#train)
* [Chat-bot building](#bot)
* [Testing chatbot](#test)

# Pre-work <a class="anchor" id="pre"></a>

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


Смерджил FirstAid и ShinyMETALBot json файла для того, чтобы у бот мог отвечать и приветствовать пользователя!

In [2]:
with open('merged_data.json') as json_data:
    intents = json.load(json_data)

In [3]:
sasha_json = json.dumps(intents, indent=3)
print(sasha_json)

{
   "intents": [
      {
         "tag": "Greeting",
         "patterns": [
            "Hi",
            "Hi there",
            "Hola",
            "Hello",
            "Hello there",
            "Hya",
            "Hya there"
         ],
         "responses": [
            "Hi human, please tell me your GeniSys user",
            "Hello human, please tell me your GeniSys user",
            "Hola human, please tell me your GeniSys user"
         ],
         "extension": {
            "function": "",
            "entities": false,
            "responses": []
         },
         "context": {
            "in": "",
            "out": "GreetingUserRequest",
            "clear": false
         },
         "entityType": "NA",
         "entities": []
      },
      {
         "tag": "GreetingResponse",
         "patterns": [
            "My user is Adam",
            "This is Adam",
            "I am Adam",
            "It is Adam",
            "My user is Bella",
            "This is Bell

- tag (Уникальное название)
- patterns (шаблоны предложений для нашего классификатора текста нейронной сети)
- responses (Один из примеров будет использоваться для ответа пользователю)

# Intents preparation <a class="anchor" id="prep"></a>

**Загрузив JSON-файл интентов, обозначим слова, документы и классы для классификации**

In [4]:
words = []
documents = []
classes = []
ignore_words = ['?']

# Проходимся по каждому предложению в шаблонах
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # токенизируем каждое слово в предложении
        w = nltk.word_tokenize(pattern)
        # добавляем в список слов words
        words.extend(w)
        # добавляем полученные документы в корпус
        documents.append((w, intent['tag']))
        # добавляем в список наших классов
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# стеммизируем и приводим к нижнему регистру каждое слово и удаляем повторяющиеся слова
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# убираем дубликаты
classes = sorted(list(set(classes)))

print(len(documents),'documents')
print(len(classes),'classes', classes)
print(len(words),'unique stemmed words', words)

331 documents
66 classes ['Abdonominal Pain', 'Abrasions', 'Broken Toe', 'Bruises', 'CPR', 'Chemical Burn', 'Choking', 'Clever', 'Cold', 'Cough', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'Cuts', 'Diarrhea', 'Drowning', 'Eye Injury', 'Fainting', 'Fever', 'Fracture', 'Frost bite', 'Gastrointestinal problems', 'GoodBye', 'Gossip', 'Greeting', 'GreetingResponse', 'Head Injury', 'Headache', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bites', 'Jokes', 'NameQuery', 'Nasal Congestion', 'Normal Bleeding', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'Poison', 'Pulled Muscle', 'Rash', 'RealNameQuery', 'Rectal bleeding', 'SelfAware', 'Shutup', 'Skin problems', 'Sore Throat', 'Splinter', 'Sprains', 'Strains', 'Sun Burn', 'Swearing', 'Teeth', 'Testicle Pain', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'Vertigo', 'WhoAmI', 'Wound', 'animal bite', 'nose bleed', 'seizure', 'snake bite', 'stings']
209 unique stemmed words ['!', "'s", ',', 'a', 'abdonom

**Создали список документов (предложений), где каждое предложение представляет собой список слов, а каждый документ связан с интентом (классом).**

# Training <a class="anchor" id="train"></a>

**Теперь преобразовываем слова в тензорное представление и обучаем нашу модель на 1000 эпохах с batch_size-8**

In [5]:
# Создаем массив с тренировочными данными
training = []
output = []
# создаем пустой массив для вывода данных
output_empty = [0] * len(classes)

# тренировочный датасет с набором слов для каждого предложения
for doc in documents:
    # инициализируем bag of words
    bag = []
    # получаем список токенизированных слов для шаблона
    pattern_words = doc[0]
    # стеммизируем каждое слово
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # создаем массив для bag of words
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # выводим 0 для каждого тега и 1 для текущего тега
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# шафлим наши фичи и преобразовываем в многомерный массив
random.shuffle(training)
training = np.array(training, dtype=object)

# Список тренировочных и тестовых данных
train_x = list(training[:,0])
train_y = list(training[:,1])

**Также шафлим наши данные, где TF будет использовать некоторые из них в качестве тестовых данных для измерения точности новой модели.**

In [6]:
# Очищаем стек графа и сбрасываем на дефолтный
tf.compat.v1.reset_default_graph()
# Создаем полносвязную нейронную сеть
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Инициализируем модель и настраиваем tensorboard для понимания структуры и производительности эксперимента
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Обучаем наши данные и применяем алгоритм градиентного спуска
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 41999  | total loss: 0.00003 | time: 0.048s
| Adam | epoch: 1000 | loss: 0.00003 - acc: 1.0000 -- iter: 328/331
Training Step: 42000  | total loss: 0.00003 | time: 0.050s
| Adam | epoch: 1000 | loss: 0.00003 - acc: 1.0000 -- iter: 331/331
--
INFO:tensorflow:C:\Users\onlym\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
%load_ext tensorboard

In [8]:
%tensorboard --logdir='tflearn_logs'

Reusing TensorBoard on port 6006 (pid 8488), started 1:24:15 ago. (Use '!kill 8488' to kill it.)

В итоге за 1000 эпох **loss: 0.00066 - acc: 1.0000**

In [9]:
# Сохранем нашю модель где-нибудь
pickle.dump({'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y},open('training_data','wb'))

# Chat-bot building <a class="anchor" id="bot"></a>

In [10]:
#Загружаем нашу модель
data = pickle.load(open('training_data', 'rb'))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\onlym\chatbot\model.tflearn


**Прежде, чем начать обработку интентов, сначала нужно создать способ набора слов из пользовательского чата:**

In [11]:
def clean_up_sentence(sentence):
     # токенизируем каждый паттерн
    sentence_words = nltk.word_tokenize(sentence)
    # стеммизируем каждое слово
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Возвращаем bar of words:0 или 1 для каждого слова в существующем наборе данных в предложении
def bow(sentence, words, show_details=False):
    # токенизируем каждый паттерн
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print('found in bag: %s' % w)

    return(np.array(bag))

In [12]:
p = bow('How to remove Splinters', words)
print(p)
print(model.predict([p]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[0.0000000e+00 0.0000000e+00 1.6181495e-11 1.3634790e-15 0.0000000e+00
  0.0000000e+00 4.5977477e-14 4.0024359e-20 1.6036110e-18 1.6994215e-09
  6.8096840e-24 1.0972958e-32 2.7475591e-35 0.0000000e+00 0.0000000e+00
  2.7886547e-21 1.0424215e-29 1.0254489e-24 0.0000000e+00 7.7402208e-27
  0.0000000e+00 0.0000000e+00 1.7959418e-22 1.7735267e-10 0.0000000e+00
  4.9376749e-35 0.0000000e+00 4.8376830e-21 9.5968262e-38 1.9375493e-35
  2.3469379e-37 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  7.3228562e-24 0.0000000e+00 6.8870121e-16 4.3088981e-22 2.8721464e-26

In [13]:
ERROR_THRESHOLD = 0.1
def classify(sentence):
    # Генерируем вероятности из модели
    results = model.predict([bow(sentence, words)])[0]
    # Фильтруем наши прогнозы на основе выставленного порога
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Сртрируем вероятности
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # Если есть классификация => метчим соответстующий тег интента
    if results:
        # Проходимся до тех пор, пока появятся совпадения для обработки
        while results:
            for i in intents['intents']:
                # Ищем тег, соответствующий первому результату
                if i['tag'] == results[0][0]:
                    # Рандомный ответ из интента
                    return print(random.choice(i['responses']))

            results.pop(0)

Каждое предложение, переданное в response(''), классифицируется. Наш классификатор использует model.predict(). Вероятности, возвращаемые моделью, выстраиваются в ряд с нашими определениями намерений, чтобы создать список потенциальных ответов.

Если одна или несколько классификаций превышают пороговое значение, проверяем, соответствует ли тег интенту и затем обрабатываем его. Будем рассматривать список классификации как стек и извлекать из стека поиск подходящего совпадения до того момента, пока не найдем его или он не будет пустым.

**Рассмотрим пример классификации, который возвращает наиболее вероятный тег и его вероятности**

**Классифицируется все прекрасно, но в большинстве тегов пропущены предполагаемые ответы (дописал парочку для примера =)**

In [14]:
classify('How do you treat abrasions?')

[('Abrasions', 0.99997747)]

In [15]:
classify('How do you treat nasal Congestion?')

[('Nasal Congestion', 0.99999964)]

In [16]:
classify('How do you treat Heat Stroke?')

[('Heat Stroke', 0.9999956)]

In [17]:
response('How do you treat a chemical burn?')

Call 911, remove dry chemicals, remove contaminated clothing or jewelry, bandage the burn and rinse again if needed.


In [18]:
response('How do you treat a Poison?')

Call 911, remove anything remaining in the person's mouth, remove any contaminated clothing using gloves. Rinse the skin for 15 to 20 minutes in a shower or with a hose, Call Poison Help at 800-222-1222 in the United States


In [19]:
response('do you take cash?')

I read you loud and clear!


# Testing chatbot <a class="anchor" id="test"></a>

**Теперь рассмотрим базовый контекст пользовательского разговора с чат-ботом по теме оказания первой помощи**

Словарь context  будет содержать состояние для каждого пользователя. Будем использовать некоторые уникальные идентификаторы для каждого пользователя (например, номер ячейки). Это позволяет поддерживать состояние для нескольких пользователей одновременно.

In [20]:
# Создадим  хранения пользовательского контекста
context = {}

ERROR_THRESHOLD = 0.1
def classify(sentence):
    # Генерируем вероятности из модели
    results = model.predict([bow(sentence, words)])[0]
    # Фильтруем наши прогнозы на основе выставленного порога
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    #  Сртрируем вероятности
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # Если есть классификация => метчим соответстующий тег интента
    if results:
        # Проходимся до тех пор, пока появятся совпадения для обработки
        while results:
            for i in intents['intents']:
                # Ищем тег, соответствующий первому результату
                if i['tag'] == results[0][0]:
                    # Устанавливаем контекст для данного интента, если нужно будет
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # Проверяем является ли этот интент контекстуальным и применим ли он к разговорю пользователя?
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # Рандомный ответ из интента
                        return print(random.choice(i['responses']))

            results.pop(0)

**Попробуем воссоздать общение пользователя с чат-ботом)**

In [21]:
response('Hi')

Hello human, please tell me your GeniSys user


In [22]:
response('This is Adam')

OK! Hola <HUMAN>, how can I help you?


In [23]:
response('Tell me a joke')

I rang up British Telecom, I said, 'I want to report a nuisance caller', he said 'Not you again'.  


In [24]:
response('Tell me a joke')

A highly excited man rang up for an ambulance. 'Quickly, come quickly', he shouted, 'My wife's about to have a baby.' 'Is this her first baby?' asked the operator. 'No, you fool', came the reply, 'It's her husband.'


In [25]:
response('I have a headache')

Give ibuprofen (Advil, Motrin), aspirin, or acetaminophen (Tylenol) for pain. Avoid ibuprofen and other NSAIDs if the person has heart failure or kidney failure. Do not give aspirin to a child under age 18.


In [26]:
response('How do you treat Diarrhea?')

1)Hydrating the body is essential for recovering from diarrhea.This causes the body to lose electrolytes such as sodium and chloride. 2)It is highly recommended to avoid dairy products, as they may worsen diarrhea in some people. 3)However, if diarrhea lasts for more than 2 days, seek medical advice to avoid complications.


In [27]:
response('Hasta la vista, baby')

Bye! Come back again soon.
